In [32]:
import math
from datetime import timedelta

import pandas as pd

In [33]:
# General restrictions:
start_date = "2025-01-01"
num_days = 30

employee_restrictions = {
    "hours_per_shift": 7,
    "max_hours_week_employee": 37.5,
    "max_hours_year_employee": 1852.5, # TODO: Pending to check
    "min_weekend_rest_month_employee": 1,
    "max_timeoff_employee": 2,
    "shifts": ["M", "T"],
    "max_persons_per_shift": {
        "M": 1,
        "T": 1,
    },
}

In [34]:
# Employees information
employees = [
    {
        "name": "E1",
        "capacity": 1,
    },
    {
        "name": "E2",
        "capacity": 1,
    },
    {
        "name": "E3",
        "capacity": 0.77,
    },
]

for one_employee in employees:
    one_employee["max_hours_year"] = math.ceil(
        employee_restrictions["max_hours_year_employee"] * one_employee["capacity"]
    )
    one_employee["max_hours_week"] = math.ceil(
        employee_restrictions["max_hours_week_employee"] * one_employee["capacity"]
    )

employees

[{'name': 'E1', 'capacity': 1, 'max_hours_year': 1853, 'max_hours_week': 38},
 {'name': 'E2', 'capacity': 1, 'max_hours_year': 1853, 'max_hours_week': 38},
 {'name': 'E3',
  'capacity': 0.77,
  'max_hours_year': 1427,
  'max_hours_week': 29}]

In [35]:
# Init employees with dates:
dates = pd.date_range(start=start_date, periods=num_days, freq="D")
employees_info = pd.DataFrame(
    index=dates,
    columns=[emp["name"] for emp in employees],
data="")
employees_info

,E1,E2,E3
2025-01-01,,,
2025-01-02,,,
2025-01-03,,,
2025-01-04,,,
2025-01-05,,,
2025-01-06,,,
2025-01-07,,,
2025-01-08,,,
2025-01-09,,,
2025-01-10,,,


In [36]:
# Init all employees by shift
all_employees_by_shift = pd.DataFrame(index=dates, columns=[one_shift for one_shift in employee_restrictions["shifts"]])
all_employees_by_shift[:] = 0
all_employees_by_shift

,M,T
2025-01-01,0,0
2025-01-02,0,0
2025-01-03,0,0
2025-01-04,0,0
2025-01-05,0,0
2025-01-06,0,0
2025-01-07,0,0
2025-01-08,0,0
2025-01-09,0,0
2025-01-10,0,0


In [37]:
# Fill information:

for date in all_employees_by_shift.index:
    for shift in all_employees_by_shift.columns:
        if all_employees_by_shift.loc[date, shift] >= employee_restrictions["max_persons_per_shift"][shift]: # No more employees needed
            continue

        for employee in employees_info.columns:
            if not employees_info.loc[date, employee]:
                seven_days_ago = date - timedelta(days=7)
                last_7_days_employee = employees_info.loc[seven_days_ago:date, employee]
                shift_counts = last_7_days_employee.value_counts().reindex(["M", "T"], fill_value=0)
                total_hours_in_7_days = shift_counts.sum()
                employee_capacity = next(emp["capacity"] for emp in employees if emp["name"] == employee)
                if total_hours_in_7_days * employee_restrictions["hours_per_shift"] >= employee_restrictions["max_hours_week_employee"] * employee_capacity:
                    continue
                all_employees_by_shift.loc[date, shift] += 1
                employees_info.loc[date, employee] = shift
                if all_employees_by_shift.loc[date, shift] >= employee_restrictions["max_persons_per_shift"][shift]: # No more employees needed
                    break


In [38]:
all_employees_by_shift.index = pd.to_datetime(all_employees_by_shift.index)
all_employees_by_shift.index = all_employees_by_shift.index.strftime("%Y-%m-%d")
all_employees_by_shift

,M,T
2025-01-01,1,1
2025-01-02,1,1
2025-01-03,1,1
2025-01-04,1,1
2025-01-05,1,1
2025-01-06,1,1
2025-01-07,1,0
2025-01-08,1,0
2025-01-09,1,1
2025-01-10,1,1


In [39]:
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info

,E1,E2,E3
2025-01-01,M,T,
2025-01-02,M,T,
2025-01-03,M,T,
2025-01-04,M,T,
2025-01-05,M,T,
2025-01-06,M,T,
2025-01-07,,,M
2025-01-08,,,M
2025-01-09,M,T,
2025-01-10,M,T,


In [40]:
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info[["E1", "E2", "E3"]]

,E1,E2,E3
2025-01-01,M,T,
2025-01-02,M,T,
2025-01-03,M,T,
2025-01-04,M,T,
2025-01-05,M,T,
2025-01-06,M,T,
2025-01-07,,,M
2025-01-08,,,M
2025-01-09,M,T,
2025-01-10,M,T,


In [41]:
output_filename = "../samples/m_a_2025.xlsx"
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info.to_excel(output_filename, sheet_name="Shift Schedule")